In [1]:
# using %matplotlib inline to immediatly draw plot after run
%matplotlib inline 

# imports
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import os
import pickle
from dotenv import load_dotenv, find_dotenv
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from typing import Optional
from sklearn.model_selection import ShuffleSplit
from sklearn.base import clone
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


"""prepare environment variable"""
load_dotenv(find_dotenv())

True

# Data Preprocessing Pipeline

In [2]:
class DataLoader:
    @staticmethod
    def load_data(path: str) -> Optional[pd.DataFrame]:
        if os.path.exists(path):
            return pd.read_csv(path)
        return None

## Data Cleaner Class 

In [3]:
class DataCleaner:
    @staticmethod
    def drop_feature(dataset: pd.DataFrame, columns: list) -> pd.DataFrame:
        dataset = dataset.copy()
        dataset = dataset.drop(columns=columns)
        return dataset

    @staticmethod
    def change_feature_type(dataset: pd.DataFrame, mapper: dict) -> pd.DataFrame:
        dataset = dataset.copy()
        dataset = dataset.astype(mapper)
        return dataset
    
    @staticmethod
    def impute_missing_numerical_feature(dataset: pd.DataFrame, dataset_type: str) -> pd.DataFrame:
        numerical_features = dataset.select_dtypes(include=['int', 'float']).columns.tolist()        
        if dataset_type == 'train':
            # fill missing numerical features with mean
            imputer = SimpleImputer(strategy='mean')
            dataset.loc[:, numerical_features] = imputer.fit_transform(dataset.loc[:, numerical_features])
            pickle.dump(imputer, open(os.getenv('NUMERICAL_IMPUTER_PATH'), "wb"))
            
        elif dataset_type == 'test':
            # fill missing numerical features with saved mean imputer
            imputer = pickle.load(open(os.getenv('NUMERICAL_IMPUTER_PATH'), "rb"))
            dataset.loc[:, numerical_features] = imputer.fit_transform(dataset.loc[:, numerical_features])
        else:
            print("dataset_type is neither train or test")
        return dataset

    @staticmethod
    def impute_missing_categorical_feature(
        dataset: pd.DataFrame, dataset_type: str, 
        categorical_features_missing_on_purpose: list = [
            'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
            'BsmtFinType2','GarageType', 'GarageFinish', 'GarageQual',
            'GarageCond','Alley', 'MasVnrType', 'Fence', 'FireplaceQu', 
            'MiscFeature', 'PoolQC'
        ]) -> pd.DataFrame:

        categorical_features = dataset.select_dtypes(include=['object']).columns.tolist()

        # fill expected missing categorical values
        imputer = SimpleImputer(strategy='constant', fill_value='Missing')
        dataset.loc[:, categorical_features_missing_on_purpose] = imputer.fit_transform(dataset.loc[:, categorical_features_missing_on_purpose])

        if dataset_type == 'train':
            # fill missing categorical features with mode
            imputer = SimpleImputer(strategy='most_frequent')
            dataset.loc[:, categorical_features] = imputer.fit_transform(dataset.loc[:, categorical_features])
            pickle.dump(imputer, open(os.getenv('CATEGORICAL_IMPUTER_PATH'), 'wb'))
            
        elif dataset_type == 'test':
            # fill missing categorical features with saved mode imputer
            imputer = pickle.load(open(os.getenv('CATEGORICAL_IMPUTER_PATH'), "rb"))
            dataset.loc[:, categorical_features] = imputer.fit_transform(dataset.loc[:, categorical_features])
        else:
            print("dataset_type is neither train or test")
            
        return dataset

## Data Feature Generator Class

In [4]:
class DataFeatureGenerator:
    @staticmethod
    def generate_feature_by_sum(dataset: pd.DataFrame, features: list, new_feature_name: str) -> pd.DataFrame:
        dataset=dataset.copy()
        dataset.loc[:, new_feature_name] = dataset.loc[:, features].sum(axis=1)
        return dataset 
        
    @staticmethod
    def generate_feature_by_mean(dataset: pd.DataFrame, features: list, new_feature_name: str) -> pd.DataFrame:
        dataset=dataset.copy()
        dataset.loc[:, new_feature_name] = dataset.loc[:, features].mean(axis=1).astype(int)
        return dataset 

# Data Feature Selector

In [5]:
class DataFeatureSelector:
    @staticmethod
    def select_feature(dataset: pd.DataFrame, features: list) -> pd.DataFrame:
        dataset = dataset.copy()
        try:
            dataset = dataset.loc[:, features]
        except Exception as e:
            print(e)
        return dataset 

# Data Feature Transformer

In [6]:
class DataFeatureTransformer:

    def log_transform_feature(dataset: pd.DataFrame) -> pd.DataFrame:
        dataset = dataset.copy()
        try:
            numerical_columns = dataset.select_dtypes(include=['int', 'float']).columns
            dataset.loc[:, numerical_columns] = np.log1p(dataset.loc[:, numerical_columns])
        except Exception as e:
            print(e)
        return dataset

    def ordinal_encoding(dataset: pd.DataFrame) -> pd.DataFrame:
        dataset = dataset.copy()
        ordinal_categorical_columns ={
            "ExterQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "ExterCond": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "BsmtQual":  ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
            "BsmtCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
            "BsmtExposure": ["Gd", "Av", "Mn", "No", "Missing"],
            "BsmtFinType1": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
            "BsmtFinType2": ["GLQ", "ALQ", "BLQ", "Rec", "LwQ", "Unf", "Missing"],
            "HeatingQC": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "KitchenQual": ["Ex", "Gd", "TA", "Fa", "Po"], 
            "FireplaceQu": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"],
            "GarageFinish": ["Fin", "RFn", "Unf", "Missing"], 
            "GarageQual": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
            "GarageCond": ["Ex", "Gd", "TA", "Fa", "Po", "Missing"], 
            "PoolQC": ["Ex", "Gd", "TA", "Fa", "Missing"], 
            "Fence": ["GdPrv", "MnPrv", "GdWo", "MnWw", "Missing"]
        } # gather ordinal categorical column
       
        ### Categorical columns transformation
        for f, v in ordinal_categorical_columns.items():
            if f in dataset.columns:
                ordinal_encoder = OrdinalEncoder(categories=[v]) # define ordinal encoder
                dataset[f] = ordinal_encoder.fit_transform(dataset[[f]]).astype(int) # ordinal encoding
        return dataset
    
    def one_hot_encoding(dataset: pd.DataFrame) -> pd.DataFrame:
        one_hot_categorical_columns = {
            "MSSubClass": ["20", "30", "40", "45",	"50", "60", "70", "75", "80", "85", "90", "120", "150", "160", "180", "190"],
            "MSZoning": ["A", "C", "FV", "I", "RH", "RL", "RP", "RM"],
            "Street": ["Pave", "Grvl"],
            "Alley": ["Missing", "Grvl", "Pave"],
            "LotShape": ["Reg", "IR1", "IR2", "IR3"],
            "LandContour": ["Lvl", "Bnk", "Low", "HLS"],
            "Utilities": ["AllPub", "NoSewr", "NoSeWa", "ELO"],
            "LotConfig": ["Inside", "FR2", "Corner", "CulDSac", "FR3"],
            "LandSlope": ["Gtl", "Mod", "Sev"],
            "Neighborhood": ["CollgCr", "Veenker", "Crawfor", "NoRidge", "Mitchel", "Somerst", "NWAmes", "OldTown", "BrkSide", "Sawyer", "NridgHt", "NAmes", "SawyerW", "IDOTRR", "MeadowV", "Edwards", "Timber", "Gilbert", "StoneBr", "ClearCr", "NPkVill", "Blmngtn", "BrDale", "SWISU", "Blueste"],
            "Condition1": ["Norm", "Feedr", "PosN", "Artery", "RRAe", "RRNn", "RRAn", "PosA", "RRNe"],
            "Condition2": ["Norm", "Artery", "RRNn", "Feedr", "PosN", "PosA", "RRAn", "RRAe", "RRNe"],
            "BldgType": ["1Fam", "2fmCon", "Duplex", "TwnhsE", "Twnhs"],
            "HouseStyle": ["2Story", "1Story", "1.5Fin", "1.5Unf", "SFoyer", "SLvl", "2.5Unf", "2.5Fin"],
            "RoofStyle": ["Gable", "Hip", "Gambrel", "Mansard", "Flat", "Shed"],
            "RoofMatl": ["CompShg", "WdShngl", "Metal", "WdShake", "Membran", "Tar&Grv", "Roll", "ClyTile"],
            "Exterior1st": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
            "Exterior2nd": ["AsbShng", "AsphShn", "BrkComm", "BrkFace", "CBlock", "CemntBd", "HdBoard",	"ImStucc", "MetalSd", "Other", "Plywood", "PreCast", "Stone", "Stucco", "VinylSd", "Wd Sdng", "WdShing"],
            "MasVnrType": ["BrkFace", "Missing", "Stone", "BrkCmn", "CBlock"],
            "Foundation": ["PConc", "CBlock", "BrkTil", "Wood", "Slab", "Stone"],
            "Heating": ["GasA", "GasW", "Grav", "Wall", "OthW", "Floor"],
            "CentralAir": ["Y", "N"],
            "Electrical": ["SBrkr", "FuseF", "FuseA", "FuseP", "Mix", "Missing"],
            "Functional": ["Typ", "Min1", "Maj1", "Min2", "Mod", "Maj2", "Sev", "Sal"],
            "GarageType": ["Attchd", "Detchd", "BuiltIn", "CarPort", "Missing", "Basment", "2Types"],
            "PavedDrive": ["Y", "N", "P"],
            "MiscFeature": ["Missing", "Shed", "Gar2", "Othr", "TenC", "Elev"],
            "SaleType": ["WD", "New", "COD", "ConLD", "ConLI", "CWD", "ConLw", "Con", "Oth", "VWD"],
            "SaleCondition": ["Normal", "Abnorml", "Partial", "AdjLand", "Alloca", "Family"]
        } # one hot encoder column

        for f, v in one_hot_categorical_columns.items():
            if f in dataset.columns:
                one_hot_encoder = OneHotEncoder(categories=[v], sparse_output=False, handle_unknown='ignore') # one hot encoder
                dataset_encoded = pd.DataFrame(one_hot_encoder.fit_transform(dataset[[f]])) # one hot encoding
                dataset_encoded.columns = one_hot_encoder.get_feature_names_out([f])
                dataset.drop(columns=[f], inplace=True)
                dataset = pd.concat([dataset, dataset_encoded], axis=1)
        return dataset
    
    def scale_data(dataset: pd.DataFrame, scaler_name: str, dataset_type: str) -> pd.DataFrame:
        dataset = dataset.copy()
        if dataset_type == "train":
            if scaler_name == "standard":
                scaler = StandardScaler()
            elif scaler_name == "minmax":
                scaler = MinMaxScaler()
            elif scaler_name == "robust":
                scaler = RobustScaler()

            dataset_scaled = scaler.fit_transform(dataset) # scale the data    
            dataset = pd.DataFrame(data=dataset_scaled, columns=dataset.columns) # create pandas dataframe
            pickle.dump(scaler, open(os.getenv('SCALER_PATH'), 'wb')) # save the scaler

        elif dataset_type == "test":
            scaler = pickle.load(open(os.getenv('SCALER_PATH'), 'rb'))
            dataset_scaled = scaler.fit_transform(dataset) # scale the data    
            dataset = pd.DataFrame(data=dataset_scaled, columns=dataset.columns) # create pandas dataframe
        return dataset

# Data Modelling 

## Data Modelling without Feature Engineering

In [7]:
# ========================================================================================
# ============================== CROSS VALIDATION ========================================
# ========================================================================================

def preprocess_data(df: pd.DataFrame, dataset_type: str) -> pd.DataFrame:

    # ========================================================================================
    # Clean data
    # ========================================================================================
    cleaned_df = DataCleaner.drop_feature(dataset=df.copy(), columns=['Id'])
    cleaned_df = DataCleaner.change_feature_type(dataset=cleaned_df, mapper={'MSSubClass': str})
    cleaned_df = DataCleaner.impute_missing_numerical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    cleaned_df = DataCleaner.impute_missing_categorical_feature(dataset=cleaned_df, dataset_type=dataset_type)

    # ========================================================================================
    # Feature selection
    # ========================================================================================
    feature_selected_df = DataFeatureSelector.select_feature(cleaned_df, 
        [
            'LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 
            'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 
            'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF'
        ] 
        + cleaned_df.select_dtypes(include=['object']).columns.tolist()
    )

    # ========================================================================================
    # Data transformation (encoding & scaling)
    # ========================================================================================
    feature_transformed_df = DataFeatureTransformer.log_transform_feature(feature_selected_df)
    feature_transformed_df = DataFeatureTransformer.ordinal_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.one_hot_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.scale_data(dataset=feature_transformed_df, scaler_name='standard', dataset_type=dataset_type)
    
    return feature_transformed_df

models = [
    LinearRegression(), Ridge(random_state=42), Lasso(random_state=42), 
    SVR(), DecisionTreeRegressor(random_state=42), RandomForestRegressor(random_state=42), 
    AdaBoostRegressor(random_state=42), GradientBoostingRegressor(random_state=42), 
    ExtraTreesRegressor(random_state=42), XGBRegressor(random_state=42), LGBMRegressor(random_state=42),
    CatBoostRegressor(random_state=42)
]

date = datetime.datetime.now().strftime('%Y%m%d')

model_names = [model.__class__.__name__ for model in models]
rmsles = []
rmses = []
maes = []
r2s = []

for model in models:
    # Load data
    # ========================================================================================
    train_df = DataLoader.load_data(os.getenv('TRAIN_PATH'))

    # prepare X and y
    # ========================================================================================
    X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()

    # perform shuffle split validation
    # ========================================================================================
    ss = ShuffleSplit(n_splits=5, random_state=42)
    rmsle, rmse, mae, r2 = [], [], [], []

    for train_index, val_index in ss.split(X_train):
        # Split based on generated index of folding
        # ========================================================================================
        X_train_fold, X_val_fold = X_train.iloc[train_index].reset_index(drop=True), X_train.iloc[train_index].reset_index(drop=True)
        y_train_fold, y_val_fold = y_train[train_index], y_train[train_index]

        # Preprocess training and validation data
        # ========================================================================================
        X_train_fold_preprocessed = preprocess_data(X_train_fold, dataset_type="train")
        X_val_fold_preprocessed = preprocess_data(X_val_fold, dataset_type="test")

        # Train
        # ========================================================================================
        cv_model = clone(model)
        cv_model.fit(X_train_fold_preprocessed, y_train_fold)

        # Predict & evaluate
        # ========================================================================================
        y_val_fold_pred = cv_model.predict(X_val_fold_preprocessed)
        y_val_fold_pred = np.where(y_val_fold_pred < 0, 0, y_val_fold_pred)
        rmsle.append(np.sqrt(mean_squared_error(np.log1p(y_val_fold), np.log1p(y_val_fold_pred))))
        rmse.append(np.sqrt(mean_squared_error(y_val_fold, y_val_fold_pred)))
        mae.append(mean_absolute_error(y_val_fold, y_val_fold_pred))
        r2.append(r2_score(y_val_fold, y_val_fold_pred))

    # Record for final summary
    # ========================================================================================
    rmsles.append(np.mean(rmsle))
    rmses.append(np.mean(rmse))
    maes.append(np.mean(mae))
    r2s.append(np.mean(r2))
    print(f"{model.__class__.__name__} done")

pd.DataFrame({
    'model': model_names,
    'rmsle': rmsles,
    'rmse': rmses,
    'mae': maes,
    'r2': r2s
}).sort_values(by=['rmse'])

LinearRegression done
Ridge done


c:\Users\USER\Documents\GitHub\house-prices-advanced-regression-techniques\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.666e+10, tolerance: 7.873e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\USER\Documents\GitHub\house-prices-advanced-regression-techniques\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.726e+09, tolerance: 8.070e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\USER\Documents\GitHub\house-prices-advanced-regression-techniques\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not c

Lasso done


KeyboardInterrupt: 

We don't pick Decision Tree Regressor and Extra Tree Regressor because it is a clear indication of overfitting.

In [20]:
# ========================================================================================
# =========================== TRAIN MODEL FOR KAGGLE =====================================
# ========================================================================================

def preprocess_data(df: pd.DataFrame, dataset_type: str) -> pd.DataFrame:

    # ========================================================================================
    # Clean data
    # ========================================================================================
    cleaned_df = DataCleaner.drop_feature(dataset=df.copy(), columns=['Id'])
    cleaned_df = DataCleaner.change_feature_type(dataset=cleaned_df, mapper={'MSSubClass': str})
    cleaned_df = DataCleaner.impute_missing_numerical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    cleaned_df = DataCleaner.impute_missing_categorical_feature(dataset=cleaned_df, dataset_type=dataset_type)

    # ========================================================================================
    # Feature selection
    # ========================================================================================
    feature_selected_df = DataFeatureSelector.select_feature(cleaned_df, 
        [
            'LotFrontage', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 
            'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 
            'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF'
        ] 
        + cleaned_df.select_dtypes(include=['object']).columns.tolist()
    )

    # ========================================================================================
    # Data transformation (encoding & scaling)
    # ========================================================================================
    feature_transformed_df = DataFeatureTransformer.log_transform_feature(feature_selected_df)
    feature_transformed_df = DataFeatureTransformer.ordinal_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.one_hot_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.scale_data(dataset=feature_transformed_df, scaler_name='standard', dataset_type=dataset_type)
    
    return feature_transformed_df

models = [
    RandomForestRegressor(random_state=42), GradientBoostingRegressor(random_state=42), 
    XGBRegressor(random_state=42), LGBMRegressor(random_state=42),
    CatBoostRegressor(random_state=42)
]

model_names = [model.__class__.__name__ for model in models]
rmses = []
maes = []
r2s = []

for model in models:
    # Load data
    # ========================================================================================
    train_df = DataLoader.load_data(os.getenv('TRAIN_PATH'))

    # prepare X and y
    # ========================================================================================
    X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()

    # perform shuffle split validation
    # ========================================================================================
    ss = ShuffleSplit(n_splits=5, random_state=42)
    rmse, mae, r2 = [], [], []

    for train_index, val_index in ss.split(X_train):
        # Load data
        # ========================================================================================
        train_df = DataLoader.load_data(os.getenv('TRAIN_PATH'))
        test_df = DataLoader.load_data(os.getenv('TEST_PATH'))

        # Prepare X and y
        # ========================================================================================
        X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()
        X_test = test_df

        # Preprocess training and validation data
        # ========================================================================================
        X_train_preprocessed = preprocess_data(X_train, dataset_type="train")

        # Train
        # ========================================================================================
        model.fit(X_train_preprocessed, y_train)

        # Predict & evaluate
        # ========================================================================================
        X_test_preprocessed = preprocess_data(X_test, dataset_type="test")
        y_pred = model.predict(X_test_preprocessed)

        # Save model
        # ========================================================================================
        pickle.dump(model, open(os.path.join(os.getenv('MODELS_FOLDER'), f'{date}_{model.__class__.__name__.lower()}_model.pkl'), 'wb'))

        # Save to submit
        # ========================================================================================
        pd.DataFrame({
            'Id': test_df['Id'],
            'SalePrice': y_pred.ravel()
        }).to_csv(os.path.join(os.getenv('PROCESSED_FOLDER'), f'{date}_submission_{model.__class__.__name__.lower()}.csv'), index=False) # save

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002262 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2765
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 147
[LightGBM] [Info] Start training from score 180921.195890
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002147 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2765
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 147
[LightGBM] [Info] Start training from score 180921.195890
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading,

# Data Modelling with Feature Engineering
Feature engineering might be an alternative to tackle multicollinearity inside each independend features. We will be using 3 best models:
1. CatBoostRegressor
2. GradientBoostingRegressor
3. LGBMRegressor

In [8]:
# ========================================================================================
# =========================== TRAIN MODEL FOR KAGGLE =====================================
# ========================================================================================

def preprocess_data(df: pd.DataFrame, dataset_type: str) -> pd.DataFrame:

    # ========================================================================================
    # Clean data
    # ========================================================================================
    cleaned_df = DataCleaner.drop_feature(dataset=df.copy(), columns=['Id'])
    cleaned_df = DataCleaner.change_feature_type(dataset=cleaned_df, mapper={'MSSubClass': str})
    cleaned_df = DataCleaner.impute_missing_numerical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    cleaned_df = DataCleaner.impute_missing_categorical_feature(dataset=cleaned_df, dataset_type=dataset_type)

    # ========================================================================================
    # Feature Engineering
    # ========================================================================================
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(cleaned_df, ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], 'OverallYear')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['1stFlrSF',	'2ndFlrSF', 'TotalBsmtSF'], 'TotalSF')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_sum(feature_engineered_df, ['OverallQual', 'OverallCond'], 'QualCond')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_sum(feature_engineered_df, ['FullBath', 'HalfBath'], 'Bath')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd'], 'TotalAbvGrd')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['BsmtFullBath', 'BsmtHalfBath'], 'BsmtBath')


    # ========================================================================================
    # Feature selection
    # ========================================================================================
    feature_selected_df = DataFeatureSelector.select_feature(feature_engineered_df, 
        [
            'LotFrontage', 'MasVnrArea', 'OverallYear', 'TotalSF', 
            'BsmtFinSF1', 'Bath', 'QualCond', 'TotalAbvGrd', 
            'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 
            'OpenPorchSF'
        ] 
        + cleaned_df.select_dtypes(include=['object']).columns.tolist()
    ) # 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'OverallQual', 'FullBath', 'TotRmsAbvGrd', 'GrLivArea', 'TotalBsmtSF'


    # ========================================================================================
    # Data transformation (encoding & scaling)
    # ========================================================================================
    feature_transformed_df = DataFeatureTransformer.log_transform_feature(feature_selected_df)
    feature_transformed_df = DataFeatureTransformer.ordinal_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.one_hot_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.scale_data(dataset=feature_transformed_df, scaler_name='standard', dataset_type=dataset_type)
    
    return feature_transformed_df

models = [
    LinearRegression(), Ridge(random_state=42), Lasso(random_state=42), 
    SVR(), DecisionTreeRegressor(random_state=42), RandomForestRegressor(random_state=42), 
    AdaBoostRegressor(random_state=42), GradientBoostingRegressor(random_state=42), 
    ExtraTreesRegressor(random_state=42), XGBRegressor(random_state=42), LGBMRegressor(random_state=42),
    CatBoostRegressor(random_state=42)
]

date = datetime.datetime.now().strftime('%Y%m%d')

model_names = [model.__class__.__name__ for model in models]
rmsles = []
rmses = []
maes = []
r2s = []

for model in models:
    # Load data
    # ========================================================================================
    train_df = DataLoader.load_data(os.getenv('TRAIN_PATH'))

    # prepare X and y
    # ========================================================================================
    X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()

    # perform shuffle split validation
    # ========================================================================================
    ss = ShuffleSplit(n_splits=5, random_state=42)
    rmsle, rmse, mae, r2 = [], [], [], []

    for train_index, val_index in ss.split(X_train):
        # Split based on generated index of folding
        # ========================================================================================
        X_train_fold, X_val_fold = X_train.iloc[train_index].reset_index(drop=True), X_train.iloc[train_index].reset_index(drop=True)
        y_train_fold, y_val_fold = y_train[train_index], y_train[train_index]

        # Preprocess training and validation data
        # ========================================================================================
        X_train_fold_preprocessed = preprocess_data(X_train_fold, dataset_type="train")
        X_val_fold_preprocessed = preprocess_data(X_val_fold, dataset_type="test")

        # Train
        # ========================================================================================
        cv_model = clone(model)
        cv_model.fit(X_train_fold_preprocessed, y_train_fold)

        # Predict & evaluate
        # ========================================================================================
        y_val_fold_pred = cv_model.predict(X_val_fold_preprocessed)
        y_val_fold_pred = np.where(y_val_fold_pred < 0, 0, y_val_fold_pred)
        rmsle.append(np.sqrt(mean_squared_error(np.log1p(y_val_fold), np.log1p(y_val_fold_pred))))
        rmse.append(np.sqrt(mean_squared_error(y_val_fold, y_val_fold_pred)))
        mae.append(mean_absolute_error(y_val_fold, y_val_fold_pred))
        r2.append(r2_score(y_val_fold, y_val_fold_pred))

    # Record for final summary
    # ========================================================================================
    rmsles.append(np.mean(rmsle))
    rmses.append(np.mean(rmse))
    maes.append(np.mean(mae))
    r2s.append(np.mean(r2))
    print(f"{model.__class__.__name__} done")

pd.DataFrame({
    'model': model_names,
    'rmsle': rmsles,
    'rmse': rmses,
    'mae': maes,
    'r2': r2s
}).sort_values(by=['rmse'])

LinearRegression done
Ridge done


c:\Users\USER\Documents\GitHub\house-prices-advanced-regression-techniques\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.214e+10, tolerance: 7.873e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\USER\Documents\GitHub\house-prices-advanced-regression-techniques\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.987e+10, tolerance: 8.070e+08
  model = cd_fast.enet_coordinate_descent(
c:\Users\USER\Documents\GitHub\house-prices-advanced-regression-techniques\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not c

Lasso done
SVR done
DecisionTreeRegressor done
RandomForestRegressor done
AdaBoostRegressor done
GradientBoostingRegressor done
ExtraTreesRegressor done
XGBRegressor done
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002556 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2151
[LightGBM] [Info] Number of data points in the train set: 1314, number of used features: 139
[LightGBM] [Info] Start training from score 180704.734399
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002082 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2141
[LightGBM] [Info] Number of data points in the train set: 1314, number of used features: 139
[LightGBM] [Info] Start tra

,model,rmsle,rmse,mae,r2
4,DecisionTreeRegressor,0.001037,140.449377,6.392694,0.999997
8,ExtraTreesRegressor,0.001037,140.449377,6.392694,0.999997
9,XGBRegressor,0.014976,2150.355998,1509.523882,0.999261
11,CatBoostRegressor,0.049067,6987.913811,5365.602981,0.992201
5,RandomForestRegressor,0.059208,11410.485052,6453.933107,0.979223
10,LGBMRegressor,0.060274,12381.307227,6354.032425,0.975540
7,GradientBoostingRegressor,0.093180,14689.914537,10841.682860,0.965554
2,Lasso,0.391579,25907.891518,17676.995600,0.892965
1,Ridge,0.391570,25909.540061,17667.646919,0.892951
0,LinearRegression,0.392543,26185.988528,17932.559385,0.890641


In [9]:
# ========================================================================================
# =========================== TRAIN MODEL FOR KAGGLE =====================================
# ========================================================================================

def preprocess_data(df: pd.DataFrame, dataset_type: str) -> pd.DataFrame:

    # ========================================================================================
    # Clean data
    # ========================================================================================
    cleaned_df = DataCleaner.drop_feature(dataset=df.copy(), columns=['Id'])
    cleaned_df = DataCleaner.change_feature_type(dataset=cleaned_df, mapper={'MSSubClass': str})
    cleaned_df = DataCleaner.impute_missing_numerical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    cleaned_df = DataCleaner.impute_missing_categorical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    
    # ========================================================================================
    # Feature Engineering
    # ========================================================================================
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(cleaned_df, ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], 'OverallYear')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['1stFlrSF',	'2ndFlrSF', 'TotalBsmtSF'], 'TotalSF')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_sum(feature_engineered_df, ['OverallQual', 'OverallCond'], 'QualCond')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_sum(feature_engineered_df, ['FullBath', 'HalfBath'], 'Bath')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd'], 'TotalAbvGrd')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['BsmtFullBath', 'BsmtHalfBath'], 'BsmtBath')


    # ========================================================================================
    # Feature selection
    # ========================================================================================
    feature_selected_df = DataFeatureSelector.select_feature(feature_engineered_df, 
        [
            'LotFrontage', 'MasVnrArea', 'OverallYear', 'TotalSF', 
            'BsmtFinSF1', 'Bath', 'QualCond', 'TotalAbvGrd', 
            'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 
            'OpenPorchSF'
        ] 
        + cleaned_df.select_dtypes(include=['object']).columns.tolist()
    ) # 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'OverallQual', 'FullBath', 'TotRmsAbvGrd', 'GrLivArea', 'TotalBsmtSF'



    # ========================================================================================
    # Data transformation (encoding & scaling)
    # ========================================================================================
    feature_transformed_df = DataFeatureTransformer.log_transform_feature(feature_selected_df)
    feature_transformed_df = DataFeatureTransformer.ordinal_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.one_hot_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.scale_data(dataset=feature_transformed_df, scaler_name='standard', dataset_type=dataset_type)
    
    return feature_transformed_df

models = [
    GradientBoostingRegressor(random_state=42), 
    LGBMRegressor(random_state=42),
    CatBoostRegressor(random_state=42)
]

date = datetime.datetime.now().strftime('%Y%m%d')
model_names = [model.__class__.__name__ for model in models]
rmses = []
maes = []
r2s = []

for model in models:
    # Load data
    # ========================================================================================
    train_df = DataLoader.load_data(os.getenv('TRAIN_PATH'))
    test_df = DataLoader.load_data(os.getenv('TEST_PATH'))

    # Prepare X and y
    # ========================================================================================
    X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()
    X_test = test_df

    # Preprocess training and validation data
    # ========================================================================================
    X_train_preprocessed = preprocess_data(X_train, dataset_type="train")

    # Train
    # ========================================================================================
    model.fit(X_train_preprocessed, y_train)

    # Predict & evaluate
    # ========================================================================================
    X_test_preprocessed = preprocess_data(X_test, dataset_type="test")
    y_pred = model.predict(X_test_preprocessed)

    # Save model
    # ========================================================================================
    pickle.dump(model, open(os.path.join(os.getenv('MODELS_FOLDER'), f'{date}_{model.__class__.__name__.lower()}_fe_model.pkl'), 'wb'))

    # Save to submit
    # ========================================================================================
    pd.DataFrame({
        'Id': test_df['Id'],
        'SalePrice': y_pred.ravel()
    }).to_csv(os.path.join(os.getenv('PROCESSED_FOLDER'), f'{date}_submission_{model.__class__.__name__.lower()}_fe.csv'), index=False) # save

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001833 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2213
[LightGBM] [Info] Number of data points in the train set: 1460, number of used features: 143
[LightGBM] [Info] Start training from score 180921.195890
Learning rate set to 0.043466
0:	learn: 76976.3468227	total: 5.08ms	remaining: 5.07s
1:	learn: 74946.9225887	total: 10.3ms	remaining: 5.15s
2:	learn: 72921.8065216	total: 15.2ms	remaining: 5.05s
3:	learn: 71080.0284108	total: 19.7ms	remaining: 4.91s
4:	learn: 69367.2939527	total: 25.2ms	remaining: 5.01s
5:	learn: 67659.8169461	total: 28.7ms	remaining: 4.75s
6:	learn: 65717.9919065	total: 33.6ms	remaining: 4.76s
7:	learn: 63923.3406941	total: 38.5ms	remaining: 4.78s
8:	learn: 62373.5877499	total: 43.2ms	remaining: 4.76s
9:	learn: 60849.1224316	total: 47.9ms	remaini

1. Using `OverallYear`, `FloorSF`, `OveralQualCond`,`TotalAbvGrd`, `TotalAbvGrd`, `HasAlley`, `HasMasVnr`, `HasFireplace`, `HasPool`, `HasFence`, `HasMiscFeature` improve CatBoostRegressor, LGBMRegressor, and GradientBoostingRegressor.

# Hypeparameter Tuning on The Best Model
The best model is CatBoostRegressor

In [57]:
# ========================================================================================
# =========================== TRAIN MODEL FOR KAGGLE =====================================
# ========================================================================================

def preprocess_data(df: pd.DataFrame, dataset_type: str) -> pd.DataFrame:

    # ========================================================================================
    # Clean data
    # ========================================================================================
    cleaned_df = DataCleaner.drop_feature(dataset=df.copy(), columns=['Id'])
    cleaned_df = DataCleaner.change_feature_type(dataset=cleaned_df, mapper={'MSSubClass': str})
    cleaned_df = DataCleaner.impute_missing_numerical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    cleaned_df = DataCleaner.impute_missing_categorical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    
    # ========================================================================================
    # Feature Engineering
    # ========================================================================================
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(cleaned_df, ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], 'OverallYear')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['1stFlrSF',	'2ndFlrSF'], 'FloorSF')
    feature_engineered_df['OveralQualCond'] = (feature_engineered_df['OverallQual'] + feature_engineered_df['OverallCond'])/2
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd'], 'TotalAbvGrd')
    feature_engineered_df[['HasAlley', 'HasMasVnr', 'HasFireplace','HasPool', 'HasFence', 'HasMiscFeature']] = feature_engineered_df[['Alley', 'MasVnrType', 'FireplaceQu','PoolQC', 'Fence', 'MiscFeature']].applymap(lambda x: 0 if x=='Missing' else 1).astype(int)


    # ========================================================================================
    # Feature selection
    # ========================================================================================
    feature_selected_df = DataFeatureSelector.select_feature(feature_engineered_df, 
        [
            'LotFrontage', 'OveralQualCond', 'MasVnrArea', 'OverallYear', 
            'FloorSF', 'BsmtFinSF1', 'TotalBsmtSF', 'Fireplaces', 
            'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'TotalAbvGrd',
            'HasAlley', 'HasMasVnr', 'HasFireplace', 'HasPool', 'HasFence', 'HasMiscFeature'
        ] 
        + cleaned_df.select_dtypes(include=['object']).columns.tolist()
    )


    # ========================================================================================
    # Data transformation (encoding & scaling)
    # ========================================================================================
    feature_transformed_df = DataFeatureTransformer.log_transform_feature(feature_selected_df)
    feature_transformed_df = DataFeatureTransformer.ordinal_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.one_hot_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.scale_data(dataset=feature_transformed_df, scaler_name='standard', dataset_type=dataset_type)
    
    return feature_transformed_df

model = CatBoostRegressor(random_state=42)


date = datetime.datetime.now().strftime('%Y%m%d')

# Load data
# ========================================================================================
train_df = DataLoader.load_data(os.getenv('TRAIN_PATH'))
test_df = DataLoader.load_data(os.getenv('TEST_PATH'))

# Prepare X and y
# ========================================================================================
X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()
X_test = test_df

# param_distribution = (
#     {
#         'learning_rate': np.arange(0, 0.1, 0.01),
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]
#     }
# )


# Preprocess training and validation data
# ========================================================================================
X_train_preprocessed = preprocess_data(X_train, dataset_type="train")

# Train
# ========================================================================================
grid = {'learning_rate': np.arange(0, 0.1, 0.01),}

randomized_search_result = model.randomized_search(grid,
                                                   X=X_train_preprocessed,
                                                   y=y_train,
                                                   )
# model.fit(X_train_preprocessed, y_train)

# # Predict & evaluate
# # ========================================================================================
# X_test_preprocessed = preprocess_data(X_test, dataset_type="test")
# y_pred = model.predict(X_test_preprocessed)







# # Save model
# # ========================================================================================
# pickle.dump(model, open(os.path.join(os.getenv('MODELS_FOLDER'), f'{date}_{model.__class__.__name__.lower()}_fe_model.pkl'), 'wb'))

# # Save to submit
# # ========================================================================================
# pd.DataFrame({
#     'Id': test_df['Id'],
#     'SalePrice': y_pred.ravel()
# }).to_csv(os.path.join(os.getenv('PROCESSED_FOLDER'), f'{date}_submission_{model.__class__.__name__.lower()}_fe.csv'), index=False) # save

0:	learn: 195961.4253915	test: 195563.8558109	best: 195563.8558109 (0)	total: 3.44ms	remaining: 3.44s
1:	learn: 194241.6935113	test: 193758.1304452	best: 193758.1304452 (1)	total: 6.63ms	remaining: 3.31s
2:	learn: 192536.9303948	test: 192051.6245639	best: 192051.6245639 (2)	total: 9.71ms	remaining: 3.23s
3:	learn: 190829.2606049	test: 190307.3709228	best: 190307.3709228 (3)	total: 12.8ms	remaining: 3.2s
4:	learn: 189156.2834371	test: 188571.6074571	best: 188571.6074571 (4)	total: 15.9ms	remaining: 3.16s
5:	learn: 187485.1845741	test: 186895.2247706	best: 186895.2247706 (5)	total: 18.8ms	remaining: 3.11s
6:	learn: 185839.5750215	test: 185223.6860770	best: 185223.6860770 (6)	total: 23ms	remaining: 3.26s
7:	learn: 184177.6900413	test: 183581.0213759	best: 183581.0213759 (7)	total: 26.7ms	remaining: 3.31s
8:	learn: 182555.1148881	test: 181888.8361634	best: 181888.8361634 (8)	total: 29.8ms	remaining: 3.29s
9:	learn: 180970.4928680	test: 180280.3963275	best: 180280.3963275 (9)	total: 33.6ms	

In [58]:
randomized_search_result

{'params': {'learning_rate': 0.06},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               

In [59]:
# ========================================================================================
# =========================== TRAIN MODEL FOR KAGGLE =====================================
# ========================================================================================

def preprocess_data(df: pd.DataFrame, dataset_type: str) -> pd.DataFrame:

    # ========================================================================================
    # Clean data
    # ========================================================================================
    cleaned_df = DataCleaner.drop_feature(dataset=df.copy(), columns=['Id'])
    cleaned_df = DataCleaner.change_feature_type(dataset=cleaned_df, mapper={'MSSubClass': str})
    cleaned_df = DataCleaner.impute_missing_numerical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    cleaned_df = DataCleaner.impute_missing_categorical_feature(dataset=cleaned_df, dataset_type=dataset_type)
    
    # ========================================================================================
    # Feature Engineering
    # ========================================================================================
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(cleaned_df, ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt'], 'OverallYear')
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['1stFlrSF',	'2ndFlrSF'], 'FloorSF')
    feature_engineered_df['OveralQualCond'] = (feature_engineered_df['OverallQual'] + feature_engineered_df['OverallCond'])/2
    feature_engineered_df = DataFeatureGenerator.generate_feature_by_mean(feature_engineered_df, ['GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd'], 'TotalAbvGrd')
    feature_engineered_df[['HasAlley', 'HasMasVnr', 'HasFireplace','HasPool', 'HasFence', 'HasMiscFeature']] = feature_engineered_df[['Alley', 'MasVnrType', 'FireplaceQu','PoolQC', 'Fence', 'MiscFeature']].applymap(lambda x: 0 if x=='Missing' else 1).astype(int)


    # ========================================================================================
    # Feature selection
    # ========================================================================================
    feature_selected_df = DataFeatureSelector.select_feature(feature_engineered_df, 
        [
            'LotFrontage', 'OveralQualCond', 'MasVnrArea', 'OverallYear', 
            'FloorSF', 'BsmtFinSF1', 'TotalBsmtSF', 'Fireplaces', 
            'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'TotalAbvGrd',
            'HasAlley', 'HasMasVnr', 'HasFireplace', 'HasPool', 'HasFence', 'HasMiscFeature'
        ] 
        + cleaned_df.select_dtypes(include=['object']).columns.tolist()
    )


    # ========================================================================================
    # Data transformation (encoding & scaling)
    # ========================================================================================
    feature_transformed_df = DataFeatureTransformer.log_transform_feature(feature_selected_df)
    feature_transformed_df = DataFeatureTransformer.ordinal_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.one_hot_encoding(feature_transformed_df)
    feature_transformed_df = DataFeatureTransformer.scale_data(dataset=feature_transformed_df, scaler_name='standard', dataset_type=dataset_type)
    
    return feature_transformed_df

model = CatBoostRegressor(
    learning_rate = 0.06,
    random_state=42)


date = datetime.datetime.now().strftime('%Y%m%d')

# Load data
# ========================================================================================
train_df = DataLoader.load_data(os.getenv('TRAIN_PATH'))
test_df = DataLoader.load_data(os.getenv('TEST_PATH'))

# Prepare X and y
# ========================================================================================
X_train, y_train = train_df.drop(columns=['SalePrice']), train_df['SalePrice'].to_numpy()
X_test = test_df
# Preprocess training and validation data
# ========================================================================================
X_train_preprocessed = preprocess_data(X_train, dataset_type="train")

# Train Model
# ========================================================================================
model.fit(X_train_preprocessed, y_train)

# Predict & evaluate
# ========================================================================================
X_test_preprocessed = preprocess_data(X_test, dataset_type="test")
y_pred = model.predict(X_test_preprocessed)


# Save model
# ========================================================================================
pickle.dump(model, open(os.path.join(os.getenv('MODELS_FOLDER'), f'{date}_{model.__class__.__name__.lower()}_hp_model.pkl'), 'wb'))

# Save to submit
# ========================================================================================
pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': y_pred.ravel()
}).to_csv(os.path.join(os.getenv('PROCESSED_FOLDER'), f'{date}_submission_{model.__class__.__name__.lower()}_hp.csv'), index=False) # save

0:	learn: 76269.1360643	total: 3.46ms	remaining: 3.46s
1:	learn: 73192.1090302	total: 6.92ms	remaining: 3.45s
2:	learn: 70409.1084627	total: 9.89ms	remaining: 3.29s
3:	learn: 68028.0502030	total: 13.1ms	remaining: 3.26s
4:	learn: 65751.3771858	total: 16.4ms	remaining: 3.26s
5:	learn: 63455.3416847	total: 19.7ms	remaining: 3.26s
6:	learn: 61324.0731349	total: 22.8ms	remaining: 3.24s
7:	learn: 59453.9456985	total: 26ms	remaining: 3.22s
8:	learn: 57575.8607205	total: 29.3ms	remaining: 3.22s
9:	learn: 55876.1945293	total: 32.5ms	remaining: 3.22s
10:	learn: 54170.3398507	total: 35.9ms	remaining: 3.23s
11:	learn: 52450.2501105	total: 38.7ms	remaining: 3.19s
12:	learn: 50878.1147194	total: 41.8ms	remaining: 3.17s
13:	learn: 49379.1573323	total: 45.1ms	remaining: 3.17s
14:	learn: 48055.5353488	total: 48.3ms	remaining: 3.17s
15:	learn: 46791.8218055	total: 51.8ms	remaining: 3.19s
16:	learn: 45584.1052720	total: 54.9ms	remaining: 3.18s
17:	learn: 44471.8177990	total: 58ms	remaining: 3.16s
18:	le